In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

import missingno as msno

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_validate

from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')


from sklearn import ensemble

pd.set_option('display.max_columns', None)

%matplotlib inline  

housing_raw = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
price = housing_raw['SalePrice'].copy()
housing_raw.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


# Initalizing the Model

In [4]:
lasso = Lasso()
ridge = Ridge()

# Separating and Imputing Values

In [5]:
df_num = housing_raw.select_dtypes(['int64','float64']).copy()
df_num.head()

,PID,GrLivArea,SalePrice,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1,909176150,856,126000,30,NaN,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0,3,2010
2,905476230,1049,139500,120,42.0,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0,2,2009
3,911128020,1001,124900,30,60.0,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0,11,2007
4,535377150,1039,114000,70,80.0,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0,5,2009
5,534177230,1665,227000,60,70.0,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0,11,2009


In [6]:
# msno.bar(df_num, sort='ascending')

In [7]:
# Selecting numerical columns
df_num = housing_raw.select_dtypes(['int64','float64']).copy()

df_num['LotFrontage'].fillna(df_num['LotFrontage'].mean(), inplace = True)

# Because there is only 1 null value for area, we just fill in with the Year the house was built
df_num.loc[df_num['GarageArea'].isnull(), 'GarageYrBlt'] = df_num.loc[df_num['GarageArea'].isnull(),'YearBuilt']
df_num['GarageArea'].fillna(housing_raw[housing_raw['GarageType']== 'Detchd']['GarageArea'].mean(), inplace = True)


df_num.drop(['SalePrice', 'PID'],inplace=True, axis = 1)
# MsnrVrArea, TotalBsmtSF, GarageCars, BsmtHalfBath, BsmtFullBath
df_num.fillna(0,inplace = True)
price = housing_raw['SalePrice'].copy()
# msno.bar(df_num)

In [8]:
# Selecting Categorical Values
# Mapping the respective categorical columns ordinally
df_cat = housing_raw.select_dtypes(['object']).copy()
df_cat.fillna('None', inplace = True)
df_cat
# Define the mapping
po_ex_ordinal_mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['ExterQual'] = df_cat['ExterQual'].map(po_ex_ordinal_mapping)
df_cat['ExterCond'] = df_cat['ExterCond'].map(po_ex_ordinal_mapping)
df_cat['KitchenQual'] = df_cat['KitchenQual'].map(po_ex_ordinal_mapping)
df_cat['HeatingQC'] = df_cat['HeatingQC'].map(po_ex_ordinal_mapping)


# Defind the mapping
none_ex6_ordinal_mapping = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['BsmtQual'] = df_cat['BsmtQual'].map(none_ex6_ordinal_mapping)
df_cat['BsmtCond'] = df_cat['BsmtCond'].map(none_ex6_ordinal_mapping)
df_cat['FireplaceQu'] = df_cat['FireplaceQu'].map(none_ex6_ordinal_mapping)
df_cat['GarageQual'] = df_cat['GarageQual'].map(none_ex6_ordinal_mapping)
df_cat['GarageCond'] = df_cat['GarageCond'].map(none_ex6_ordinal_mapping)


# Define the mapping
none_fin_ordinal_mapping = {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin':3}
# Applying the mapping
df_cat['GarageFinish'] = df_cat['GarageFinish'].map(none_fin_ordinal_mapping)


# Define the mapping
none_ex5_ordinal_mapping = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd':3, 'Ex':4}
# Applying the mapping
df_cat['PoolQC'] = df_cat['PoolQC'].map(none_ex5_ordinal_mapping)

# Define mapping
binary_mapping = {'Y': 1, 'N':0}
df_cat['CentralAir'] = df_cat['CentralAir'].map(binary_mapping)


df_cat.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
1,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,CBlock,3,3,No,Rec,Unf,GasA,3,1,SBrkr,3,Typ,4,Detchd,1,3,3,Y,0,None,None,WD,Normal
2,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,4,3,CBlock,4,3,Mn,GLQ,ALQ,GasA,3,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,WD,Normal
3,C (all),Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,4,3,BrkTil,3,3,No,ALQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Detchd,1,3,1,N,0,None,None,WD,Normal
4,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,Gable,CompShg,MetalSd,MetalSd,None,4,4,BrkTil,2,3,No,Unf,Unf,GasA,4,1,SBrkr,3,Typ,0,Detchd,1,3,3,N,0,None,None,WD,Normal
5,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,4,3,PConc,4,3,No,GLQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,WD,Normal


In [9]:
# msno.bar(df_cat)

In [10]:
df = pd.concat([df_num, df_cat], axis = 1)
df

,GrLivArea,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
1,856,30,68.516053,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0,3,2010,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,CBlock,3,3,No,Rec,Unf,GasA,3,1,SBrkr,3,Typ,4,Detchd,1,3,3,Y,0,None,None,WD,Normal
2,1049,120,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0,2,2009,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,4,3,CBlock,4,3,Mn,GLQ,ALQ,GasA,3,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,WD,Normal
3,1001,30,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0,11,2007,C (all),Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,4,3,BrkTil,3,3,No,ALQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Detchd,1,3,1,N,0,None,None,WD,Normal
4,1039,70,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0,5,2009,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,Gable,CompShg,MetalSd,MetalSd,None,4,4,BrkTil,2,3,No,Unf,Unf,GasA,4,1,SBrkr,3,Typ,0,Detchd,1,3,3,N,0,None,None,WD,Normal
5,1665,60,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0,11,2009,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,4,3,PConc,4,3,No,GLQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,30,68.516053,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,952.0,952,0,0,0.0,0.0,1,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,0,5,2009,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,BrkTil,3,3,No,Unf,Unf,Grav,2,0,FuseF,2,Typ,4,Detchd,1,2,1,P,0,None,None,WD,Normal
760,1733,20,68.516053,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0.0,1733,0,0,0.0,0.0,2,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,0,6,2009,RL,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,Hip,CompShg,BrkFace,Wd Sdng,None,3,3,Slab,0,0,None,None,None,GasA,5,1,FuseA,3,Min2,4,Attchd,1,3,3,Y,0,None,None,WD,Normal
761,2002,90,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,717.0,1001.0,1001,1001,0,0.0,0.0,2,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,0,8,2007,RH,Pave,None,Reg,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,Duplex,2Story,Gable,CompShg,MetalSd,MetalSd,None,3,3,CBlock,3,3,No,BLQ,Unf,GasA,3,0,FuseA,3,Typ,0,2Types,1,3,3,Y,0,None,None,WD,Normal
762,1842,60,68.516053,8826,7,5,2000,2000,144.0,841.0,0.0,144.0,985.0,985,857,0,1.0,0.0,2,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,0,7,2007,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,V

In [11]:
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(include = ['int64','float64']).columns

In [12]:
print('Num cat cols:', len(cat_cols))
print('Num num cols:', len(num_cols))

Num cat cols: 31
Num num cols: 48


In [13]:
x = df
y = price

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=42)

In [15]:
# Create the preprocessing pipelines for both numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # This will standardize the continuous features
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'first'))  # This will one-hot encode the categorical features
])

# Combine the numeric and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

# Create a pipeline that combines the preprocessor with a regression model
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso())
    
])

ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
    
])
lasso_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['GrLivArea', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'BsmtFullBath', 'Bsmt...
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Electrical', 'Functional',
       'GarageType', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType',
       'SaleCondition'],
      dtype='object'))])),
                ('regressor', Lasso())])

# GridSearch

In [16]:
np.linspace(0.001,200, 100)

array([1.00000000e-03, 2.02119192e+00, 4.04138384e+00, 6.06157576e+00,
       8.08176768e+00, 1.01019596e+01, 1.21221515e+01, 1.41423434e+01,
       1.61625354e+01, 1.81827273e+01, 2.02029192e+01, 2.22231111e+01,
       2.42433030e+01, 2.62634949e+01, 2.82836869e+01, 3.03038788e+01,
       3.23240707e+01, 3.43442626e+01, 3.63644545e+01, 3.83846465e+01,
       4.04048384e+01, 4.24250303e+01, 4.44452222e+01, 4.64654141e+01,
       4.84856061e+01, 5.05057980e+01, 5.25259899e+01, 5.45461818e+01,
       5.65663737e+01, 5.85865657e+01, 6.06067576e+01, 6.26269495e+01,
       6.46471414e+01, 6.66673333e+01, 6.86875253e+01, 7.07077172e+01,
       7.27279091e+01, 7.47481010e+01, 7.67682929e+01, 7.87884848e+01,
       8.08086768e+01, 8.28288687e+01, 8.48490606e+01, 8.68692525e+01,
       8.88894444e+01, 9.09096364e+01, 9.29298283e+01, 9.49500202e+01,
       9.69702121e+01, 9.89904040e+01, 1.01010596e+02, 1.03030788e+02,
       1.05050980e+02, 1.07071172e+02, 1.09091364e+02, 1.11111556e+02,
      

In [17]:
param_grid = {
    'regressor__alpha': np.linspace(0.001,300, 100)
}
grid_search = GridSearchCV(lasso_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['GrLivArea', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQua...
       2.30303263e+02, 2.33333556e+02, 2.36363848e+02, 2.39394141e+02,
       2.42424434e+02, 2.45454727e+02, 2.48485020e+02, 2.51515313e+02,
       2.54545606e+02, 2.57575899e+02, 2.60606192e+02, 2.63636485e+02,
       2.66666778e+02, 2.69697071e+02, 2.72727364e+02, 2.75757657e+02,
       2.78787949e+02, 2.81818242e+02, 2.84848535e+02, 2.87878828e+02,
       2.90909121e+02, 2.93939414e+02, 2.96969707e+02, 3.00000000e+02])},
             scoring='neg_mean_squared_error')

In [18]:
best_lasso = grid_search.best_estimator_
print(best_lasso)
best_lasso.score(x_test, y_test)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['GrLivArea', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'BsmtFullBath', 'Bsmt...
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Electrical', 'Functional',
       'GarageType', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType',
       'SaleCondition'],
      dtype='object'))])),
   

0.8999948474353259

In [20]:
# This is the best score for 1 pipeline
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_parameters)
print("Best Score:", best_score)

Best Parameters: {'regressor__alpha': 200.00033333333334}
Best Score: -603236074.3104432


In [21]:
best_pipeline = grid_search.best_estimator_
best_lasso_model = best_pipeline.named_steps['regressor']
lasso_coefficients = best_lasso_model.coef_
print("Lasso Coefficients:", lasso_coefficients)

Lasso Coefficients: [25110.18027073 -5661.69029471  1333.35221991  3242.99644273
 12940.22069669  6321.10595518  7228.84395186   612.55981707
  6368.30799939  7119.30715874  2134.88996486    -0.
  6113.99416161     0.           290.40603932  -367.07769595
  1105.93261204     0.             0.           415.87611063
 -2412.07818556 -2067.31753278  1150.74812077  3492.87733343
 -1812.93345369  2090.02898802  4791.02015777   787.47593454
  -496.7718021    138.61989654     0.          2645.5317735
 -3044.04609339    67.00730435  -702.99980223  -266.61979944
  4659.73932472  -270.73850902  3572.07826368 -2155.21957076
  1445.77610577  -992.76419499  4068.62189962    -0.
    92.09652006     0.          -237.13438684  4269.08984089
     0.             0.            -0.             0.
 -2223.96115704     0.             0.             0.
     0.            -0.          -135.56618297   728.58314035
    -0.             0.            -0.           195.8681445
    -0.            -0.             0. 

In [22]:
preprocessor1 = best_pipeline.named_steps['preprocessor']
transformed_feature_names = []

# Iterate through each transformer in the ColumnTransformer
for transformer_name, transformer, columns in preprocessor1.transformers_:
    if transformer_name != 'remainder':
        if hasattr(transformer, 'get_feature_names_out'):
            # For transformers with get_feature_names_out (like one-hot encoder)
            names = transformer.get_feature_names_out(columns)
            transformed_feature_names.extend(names)
        else:
            # For transformers without get_feature_names_out (like simple imputers)
            transformed_feature_names.extend(columns)

In [23]:
feature_coefficient_pairs = zip(transformed_feature_names, lasso_coefficients)
feature_coefficient_pairs_list = list(feature_coefficient_pairs)
filtered_list = [t for t in feature_coefficient_pairs_list if t[1] != 0]
print(len(filtered_list))
filtered_list

76


[('GrLivArea', 25110.180270734236),
 ('MSSubClass', -5661.690294709159),
 ('LotFrontage', 1333.3522199075148),
 ('LotArea', 3242.996442731265),
 ('OverallQual', 12940.220696690867),
 ('OverallCond', 6321.105955183389),
 ('YearBuilt', 7228.843951861626),
 ('YearRemodAdd', 612.5598170690159),
 ('MasVnrArea', 6368.3079993895035),
 ('BsmtFinSF1', 7119.3071587436925),
 ('BsmtFinSF2', 2134.8899648623565),
 ('TotalBsmtSF', 6113.994161614998),
 ('2ndFlrSF', 290.4060393246488),
 ('LowQualFinSF', -367.0776959464114),
 ('BsmtFullBath', 1105.9326120427609),
 ('HalfBath', 415.87611062698295),
 ('BedroomAbvGr', -2412.078185557584),
 ('KitchenAbvGr', -2067.317532784766),
 ('TotRmsAbvGrd', 1150.748120770427),
 ('Fireplaces', 3492.877333429783),
 ('GarageYrBlt', -1812.9334536857034),
 ('GarageCars', 2090.0289880245164),
 ('GarageArea', 4791.020157765505),
 ('WoodDeckSF', 787.4759345404899),
 ('OpenPorchSF', -496.771802104408),
 ('EnclosedPorch', 138.61989654271417),
 ('ScreenPorch', 2645.531773500758),

In [67]:
# cv_scores = cross_val_score(lasso_pipeline, X, y, cv=5)
# print('Individual scores:', cv_scores)
# print('Mean score:', round(np.mean(cv_scores),5))

In [68]:
# lasso_pipeline.fit(X,y)

In [33]:
# def get_feature_names(column_transformer):
#     output_features = []

#     for name, pipe, features in column_transformer.transformers_:
#         if name != 'remainder':
#             for i in pipe.get_feature_names_out():
#                 output_features.append(f"{name}__{i}")
#         else:
#             output_features.extend(features)

#     return output_features


In [20]:
# results = []

In [21]:
# for params in ParameterGrid(param_grid):
#     # Setting the parameters for the pipeline
#     lasso_pipeline.set_params(**params)
    
#     # Performing cross-validation
#     cv_results = cross_validate(lasso_pipeline, X, y, cv=5, return_estimator=True, scoring='r2')
    
#     # Extracting the coefficients for each fold and storing them with the parameters
#     for i, estimator in enumerate(cv_results['estimator']):
#         fold_coefficients = estimator.named_steps['regressor'].coef_
#         results.append({
#             'params': params,
#             'fold': i,
#             'coefficients': fold_coefficients,
#             'score': cv_results['test_score'][i]
#         })

In [22]:
# feature_names = X.columns

In [23]:
# for result in results:
#     print(f"Params: {result['params']}, Fold: {result['fold']}")
#     for name, coef in zip(feature_names, result['coefficients']):
#         print(f"  {name}: {coef}")
#     print(f"Score: {result['score']}")
#     print("-" * 80)

Params: {'regressor__alpha': 0.001}, Fold: 0
  PID: 2033.9505865124033
  GrLivArea: 41502.00619592449
  MSSubClass: -3537.733452327078
  LotFrontage: 351.88252741893626
  LotArea: 5007.453065583806
  OverallQual: 10102.321962612406
  OverallCond: 5420.285612812194
  YearBuilt: 7319.39579469845
  YearRemodAdd: 1241.875959627242
  MasVnrArea: 6167.064680033012
  BsmtFinSF1: 14980.133121250152
  BsmtFinSF2: 5304.426443682526
  BsmtUnfSF: 5363.032135569351
  TotalBsmtSF: 3541.2992292381114
  1stFlrSF: -13987.074745136026
  2ndFlrSF: -8711.308798355922
  LowQualFinSF: -1796.7919795639707
  BsmtFullBath: 373.7692857473138
  BsmtHalfBath: 530.1200768477839
  FullBath: 695.8318866875065
  HalfBath: 347.33652280081026
  BedroomAbvGr: -3849.167370122063
  KitchenAbvGr: -1783.516796573742
  TotRmsAbvGrd: 1823.0408106052087
  Fireplaces: 3679.600917227805
  GarageYrBlt: -2614.8172371747432
  GarageCars: 2328.368661175003
  GarageArea: 2992.7592478543716
  WoodDeckSF: 414.5510221172873
  OpenPorchS

In [24]:
# alpha_scores = {}

In [25]:
# for result in results:
#     alpha = result['params']['regressor__alpha']
#     score = result['score']
#     if alpha not in alpha_scores:
#         alpha_scores[alpha] = []
#     alpha_scores[alpha].append(score)

In [26]:
# average_scores = {alpha: np.mean(scores) for alpha, scores in alpha_scores.items()}


In [27]:
# for alpha, avg_score in average_scores.items():
#     print(f"Alpha: {alpha}, Average Score: {avg_score}")

Alpha: 0.001, Average Score: 0.8988216210365593
Alpha: 1.011090909090909, Average Score: 0.8996895344378547
Alpha: 2.021181818181818, Average Score: 0.9003193152872255
Alpha: 3.0312727272727273, Average Score: 0.9008195500026787
Alpha: 4.041363636363637, Average Score: 0.9013158860018657
Alpha: 5.051454545454546, Average Score: 0.9017762561556328
Alpha: 6.061545454545455, Average Score: 0.9021745866997559
Alpha: 7.071636363636364, Average Score: 0.9025238757913477
Alpha: 8.081727272727273, Average Score: 0.9028506731085482
Alpha: 9.091818181818182, Average Score: 0.9031681335295957
Alpha: 10.101909090909091, Average Score: 0.90346374933046
Alpha: 11.112, Average Score: 0.9037184153189584
Alpha: 12.12209090909091, Average Score: 0.9039335727684646
Alpha: 13.132181818181818, Average Score: 0.9041472977168444
Alpha: 14.142272727272728, Average Score: 0.9043277358367099
Alpha: 15.152363636363637, Average Score: 0.9044832054327954
Alpha: 16.162454545454548, Average Score: 0.9046196330680024

In [28]:
# fold_scores = {}

# # Populate fold_scores with alpha values and their corresponding scores
# for result in results:
#     fold = result['fold']
#     alpha = result['params']['regressor__alpha']
#     score = result['score']

#     if fold not in fold_scores:
#         fold_scores[fold] = []

#     fold_scores[fold].append((alpha, score))

# # Find the best alpha for each fold
# best_alpha_per_fold = {}

# for fold, scores in fold_scores.items():
#     best_alpha = max(scores, key=lambda x: x[1])[0]  # Get alpha with max score
#     best_alpha_per_fold[fold] = best_alpha

# best_alpha_per_fold

{0: 26.26336363636364, 1: 19.192727272727275, 2: 100.0, 3: 11.112, 4: 66.667}

In [29]:
# max_alpha = max(average_scores, key=average_scores.get)
# max_score = average_scores[max_alpha]
# max_alpha,max_score

(31.313818181818185, 0.9054731090482786)

In [40]:
# model_alpha = grid_search.cv_results_['param_regressor__alpha']
# model_score = grid_search.cv_results_['mean_test_score']
# model_coef = 

In [41]:
# Get the alpha for the 5 folds 
